In [1]:
import os
import numpy as np
import pandas as pd
import collections
import torch
import networkx as nx
from ortools.sat.python import cp_model
from matplotlib import pyplot as plt

import warnings
warnings.filterwarnings("ignore")
from ortools_scheduler import ORtools_scheduler

#### ortools 实验结果
max_time = 300s

In [2]:
instances = ["ft06", "la01", "la06", "la11", "la21", "la31", "la36", "orb01", "swv01", "swv06", "swv11", "yn1", 
            "swv12", "swv13", "swv14", "swv15"]

In [7]:
# 用ortools优化，保存调度结果
origin_optimal_list = []
for instance in instances:
    scheduler = ORtools_scheduler(instance_name=instance, max_time=300)
    scheduler.optimize()
    assert scheduler.obj_val == scheduler.compute_makespan()
    print(f"For instance {instance}, obj_val in 300s is {scheduler.obj_val}")
    scheduler.store_solution()
    if scheduler.status == cp_model.OPTIMAL:
        origin_optimal_list.append(instance)
df = pd.DataFrame({'instances':origin_optimal_list})
df.to_csv("sols/origin_optimal.csv")

For instance ft06, obj_val in 300s is 55.0
For instance la01, obj_val in 300s is 666.0
For instance la06, obj_val in 300s is 926.0
For instance la11, obj_val in 300s is 1222.0
For instance la21, obj_val in 300s is 1046.0
For instance la31, obj_val in 300s is 1784.0
For instance la36, obj_val in 300s is 1268.0
For instance orb01, obj_val in 300s is 1059.0
For instance swv01, obj_val in 300s is 1429.0
For instance swv06, obj_val in 300s is 1712.0
For instance swv11, obj_val in 300s is 3207.0
For instance yn1, obj_val in 300s is 898.0
For instance swv12, obj_val in 300s is 3237.0
For instance swv13, obj_val in 300s is 3277.0
For instance swv14, obj_val in 300s is 3142.0
For instance swv15, obj_val in 300s is 3047.0


In [8]:
# 读取调度结果，计算最大加工时间
for instance in instances:
    scheduler = ORtools_scheduler(instance_name=instance, max_time=300)
    scheduler.read_solution()
    makespan = scheduler.compute_makespan()
    print(f"For instance {instance}, makespan computed is {makespan}")

For instance ft06, makespan computed is 55
For instance la01, makespan computed is 666
For instance la06, makespan computed is 926
For instance la11, makespan computed is 1222
For instance la21, makespan computed is 1046
For instance la31, makespan computed is 1784
For instance la36, makespan computed is 1268
For instance orb01, makespan computed is 1059
For instance swv01, makespan computed is 1429
For instance swv06, makespan computed is 1712
For instance swv11, makespan computed is 3207
For instance yn1, makespan computed is 898
For instance swv12, makespan computed is 3237
For instance swv13, makespan computed is 3277
For instance swv14, makespan computed is 3142
For instance swv15, makespan computed is 3047


#### 工时波动，ortools重新求解

In [ ]:
instances = ["ft06", "la01", "la06", "la11", "la21", "la31", "la36", "orb01", "yn1", "swv01", "swv06", "swv11", 
            "swv12", "swv13", "swv14", "swv15"]

In [2]:
test_instances = ["swv12", "swv13"]     # 12需要重新跑
random_rate = 0.5
cv = 0.2
n = 10

for instance in test_instances:
    directory = f"sols/{instance}/p{random_rate}cv{cv}"         # 某个样例，给定随机参数
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    if_optimal = []
    obj_vals = []
    for i in range(n):                                          # 随机生成 n 个波动工时矩阵
        scheduler = ORtools_scheduler(instance_name=instance, max_time=300)
        times = scheduler.shifted_time_(random_rate=random_rate, cv=cv)
        scheduler.shift_time(times)
        scheduler.optimize()
        obj_val = scheduler.obj_val
        obj_vals.append(obj_val)

        scheduler.store_time_mat(os.path.join(directory, f"{i}.npy"))
        #scheduler.store_solution(os.path.join(directory, f"{i}.csv"))        不需要波动后的解，只需要波动后的时间矩阵，原始的ortools解是需要的
        if scheduler.status == cp_model.OPTIMAL:
            if_optimal.append(1)
        else:
            if_optimal.append(0)
    df = pd.DataFrame({'optimal':if_optimal, "obj_val":obj_vals})
    df.to_csv(os.path.join(directory, "info.csv"))